In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.transform import *
from local.data.pipeline import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp tabular.core

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## TabularProc -

In [ ]:
#export
@docs
class TabularProc():
    "Base class to write a tabular processor for dataframes"
    order = 0
    def __init__(self, cat_names=None, cont_names=None, func=None):
        self.cat_names,self.cont_names = L(cat_names),L(cont_names)
        if func is not None: self.setup,self.__call__ = func,func
    
    def setup(self, df, trn_idx=None): pass
    def __call__(self, df):  pass

    _docs = dict(setup="Use `df` to set its state, with just `trn_idx` if passed",
                 __call__="Process `df` with the state computed during setup`")

We use this class to preprocess tabular data. `cat_names` should contain the names of the categorical variables in your dataframe, `cont_names` the names of the continuous variables. If you don't need any state, you can initiliaze a `TabularProc` with a `func` to be applied on the dataframes. Otherwise you should subclass and implement `setup` and `__call__`.

In [ ]:
show_doc(TabularProc.setup)

<h4 id="<code>TabularProc.setup</code>" class="doc_header"><code>TabularProc.setup</code><a href="https://github.com/fastai/fastai_docs/tree/master/dev/__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularProc.setup</code>(**`df`**, **`trn_idx`**=*`None`*)

Use `df` to set its state, with just `trn_idx` if passed

In [ ]:
show_doc(TabularProc.__call__)

<h4 id="<code>TabularProc.__call__</code>" class="doc_header"><code>TabularProc.__call__</code><a href="https://github.com/fastai/fastai_docs/tree/master/dev/__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularProc.__call__</code>(**`df`**)

Process `df` with the state computed during setup`

In [ ]:
#export
from pandas.api.types import is_numeric_dtype, is_categorical_dtype

In [ ]:
#export
class Categorify(TabularProc):
    "Transform the categorical variables to that type."
    order = 1
    def setup(self, df, trn_idx=None):
        self.categories = {}
        for n in self.cat_names: 
            col = df[n] if trn_idx is None else df.loc[trn_idx, n]
            self.categories[n] = pd.Categorical(col, ordered=True).categories
        
    def __call__(self, df):
        for n in self.cat_names:
            df.loc[:,n] = pd.Categorical(df.loc[:,n], categories=self.categories[n], ordered=True)

In [ ]:
show_doc(Categorize, title_level=3)

<h3 id="<code>class</code> <code>Categorize</code>" class="doc_header"><code>class</code> <code>Categorize</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_core.ipynb#Categorize--" class="source_link" style="float:right">[source]</a></h3>

> <code>Categorize</code>(**\*`args`**, **\*\*`kwargs`**) :: [`ItemTransform`](/transforms.html#ItemTransform)

Reversible transform of category string to `vocab` id

In [ ]:
cat = Categorify(cat_names='a')
df = pd.DataFrame({'a':[0,1,2,0,2]})
cat.setup(df)
test_eq(list(cat.categories['a']), [0,1,2])
cat(df)
test_eq(df['a'].cat.codes.values, np.array([0,1,2,0,2]))
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})
cat(df1)
#Values that weren't in the training df are sent to -1 (na)
test_eq(df1['a'].cat.codes.values, np.array([1,0,-1,-1,2]))

In [ ]:
cat = Categorify(cat_names='a')
df = pd.DataFrame({'a':[0,1,2,3,2]})
cat.setup(df, trn_idx=[0,1,2])
test_eq(list(cat.categories['a']), [0,1,2])
cat(df)
test_eq(df['a'].cat.codes.values, np.array([0,1,2,-1,2]))

In [ ]:
cat.categories['a']

Int64Index([0, 1, 2], dtype='int64')

In [ ]:
#export
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, df, trn_idx=None):
        self.means,self.stds = {},{}
        for n in self.cont_names:
            assert is_numeric_dtype(df[n]), (f"""Cannot normalize '{n}' column as it isn't numerical.
                Are you sure it doesn't belong in the categorical set of columns?""")
            col = (df[n] if trn_idx is None else df.loc[trn_idx,n]).values
            self.means[n],self.stds[n] = col.mean(),col.std()
    
    def __call__(self, df):
        for n in self.cont_names: df[n] = (df[n]-self.means[n]) / (1e-7 + self.stds[n])

In [ ]:
show_doc(Normalize, title_level=3)

<h3 id="<code>class</code> <code>Normalize</code>" class="doc_header"><code>class</code> <code>Normalize</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_core.ipynb#Normalization--" class="source_link" style="float:right">[source]</a></h3>

> <code>Normalize</code>(**`cat_names`**=*`None`*, **`cont_names`**=*`None`*, **`func`**=*`None`*) :: `TabularProc`

Normalize the continuous variables.

In [ ]:
norm = Normalize(cont_names='a')
df = pd.DataFrame({'a':[0,1,2,3,4]})
norm.setup(df)
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means, {'a': m})
test_close(norm.stds['a'], s)
norm(df)
test_close(df['a'].values, (x-m)/s)
df1 = pd.DataFrame({'a':[5,6,7]})
norm(df1)
test_close(df1['a'].values, (np.array([5,6,7])-m)/s)

In [ ]:
norm = Normalize(cont_names='a')
df = pd.DataFrame({'a':[0,1,2,3,4]})
norm.setup(df, trn_idx=[0,1,2])
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means, {'a': m})
test_close(norm.stds['a'], s)
norm(df)
test_close(df['a'].values, (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
mk_class('FillStrategy', **{o:o for o in ['median', 'constant', 'most_common']})
FillStrategy.__doc__ = "Namespace containing the various filling strategies"

In [ ]:
#export
_all_ = ['FillStrategy']

In [ ]:
show_doc(FillStrategy, title_level=3)

<h3 id="<code>class</code> <code>FillStrategy</code>" class="doc_header"><code>class</code> <code>FillStrategy</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillStrategy</code>(**\*`args`**, **\*\*`kwargs`**)

Namespace containing the various filling strategies

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, cat_names=None, cont_names=None, fill_strategy=FillStrategy.median, add_col=True, fill_val=0.):
        super().__init__(cat_names, cont_names)
        self.fill_strategy,self.add_col,self.fill_val = fill_strategy,add_col,fill_val
    
    def setup(self, df, trn_idx=None):
        self.na_dict = {}
        for n in self.cont_names:
            col = df[n] if trn_idx is None else df.loc[trn_idx,n]
            if pd.isnull(col).sum():
                if self.fill_strategy == FillStrategy.median: filler = col.median()
                elif self.fill_strategy == FillStrategy.constant: filler = self.fill_val
                else: filler = col.dropna().value_counts().idxmax()
                self.na_dict[n] = filler
                if self.add_col:
                    df[n+'_na'] = pd.isnull(df[n])
                    if n+'_na' not in self.cat_names: self.cat_names.append(n+'_na')

    def __call__(self, df):
        for n in self.cont_names:
            if n in self.na_dict:
                if self.add_col: df[n+'_na'] = pd.isnull(df[n])
                df[n] = df[n].fillna(self.na_dict[n])
            elif pd.isnull(df[n]).sum() != 0:
                raise Exception(f"""There are nan values in field {n} but there were none in the training set given at setup. 
                Please fix those manually.""")

In [ ]:
show_doc(FillMissing, title_level=3)

<h3 id="<code>class</code> <code>FillMissing</code>" class="doc_header"><code>class</code> <code>FillMissing</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissing</code>(**`cat_names`**=*`None`*, **`cont_names`**=*`None`*, **`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_val`**=*`0.0`*) :: `TabularProc`

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissing(cont_names='a', fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.most_common])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
fill1.setup(df); fill2.setup(df1); fill3.setup(df2)
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})
for f in [fill1, fill2, fill3]: test_eq(f.cat_names, ['a_na'])

fill1(df); fill2(df1); fill3(df2)
for df_,v in zip([df, df1, df2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(df_['a_na'].values, np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
dfa1 = dfa.copy(); dfa2 = dfa.copy()
fill1(dfa); fill2(dfa1); fill3(dfa2)
for df_,v in zip([dfa, dfa1, dfa2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([v, 0, v]))
    test_eq(df_['a_na'].values, np.array([1, 0, 1]))

## TabularProcessor -

In [ ]:
#export
class TabularPreprocessor():
    "An object that will preprocess dataframes using `procs`"
    def __init__(self, procs, cat_names=None, cont_names=None, inplace=True):
        self.cat_names,self.cont_names,self.inplace = L(cat_names),L(cont_names),inplace
        self.procs = L(p if isinstance(p, type) else partial(TabularProc, func=p) for p in procs).sorted(key='order')
    
    def __call__(self, df, trn_idx=None):
        "Call each of `self.procs` on `df`, setup on `df[trn_idx]` if not None"
        df = df if self.inplace else df.copy()
        if trn_idx is None:
            for p in self.procs: p(df)
        else:
            self.procs,procs = [],self.procs
            for p in procs: 
                p_ = p(cat_names=self.cat_names, cont_names=self.cont_names)
                p_.setup(df, trn_idx=trn_idx)
                p_(df)
                self.cat_names,self.cont_names = p_.cat_names,p_.cont_names
                self.procs.append(p_)
            self.classes = {n:'#na#'+L(df[n].cat.categories, use_list=True) for n,c in df[self.cat_names].items()}
            for p in self.procs:
                if isinstance(p, Normalize): self.means,self.stds = p.means,p.stds
        return df

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
proc = TabularPreprocessor(procs, 'a', 'b', inplace=False)

#Test reordering and partialize
test_eq(proc.procs, [FillMissing, proc.procs[1], Categorify, Normalize])
test_eq(proc.procs[1].func, TabularProc)
test_eq(proc.procs[1].keywords, {'func': noop})

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})

#Test setup and apply on df_trn
df1 = proc(df, trn_idx=range_of(df))
test_eq(df1['a'].cat.codes, [0,1,2,1,1,2,0])
test_eq(df1['b_na'].cat.codes, [0,0,1,0,0,0,0])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(df1['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

#Test apply on df_val
df = pd.DataFrame({'a':[2,1,3], 'b':[4,5,np.nan]})
df1 = proc(df)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})
test_eq(df1['a'].cat.codes, [2,1,-1])
test_eq(df1['b_na'].cat.codes, [0,0,1])
x = np.array([4, 5, 1.5])
test_close(df1['b'].values, (x-m)/s)

In [ ]:
#export
def process_df(df, splitter, procs, cat_names=None, cont_names=None, inplace=True):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    proc = TabularPreprocessor(procs, cat_names, cont_names, inplace=inplace)
    splits = splitter(df)
    res = proc(df, trn_idx=splits[0])
    return res,proc

Pass the same `splitter` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
#export
class TabularLine(pd.Series):
    
    def show(self, ctx=None, **kwargs):
        if ctx is None: return self
        else: return ctx.append(self)

In [ ]:
#export
from IPython.display import display, HTML

class TensorTabular(tuple):
    
    def get_ctxs(self, max_samples=10, **kwargs):
        n_samples = min(self[0].shape[0], max_samples)
        df = pd.DataFrame({'index': range(n_samples)})
        return [df.iloc[i] for i in range(n_samples)]
    
    def display(self, ctxs):
        df = pd.DataFrame(ctxs)
        with pd.option_context('display.max_colwidth', -1): 
            display(HTML(df.to_html(index=False)))

In [ ]:
#export
class ReadTabLine(ItemTransform):
    def __init__(self, df, proc): self.df,self.proc = df,proc
    
    def encodes(self, i): 
        cats = [self.df.loc[[i], n].cat.codes[i]+1 for n in self.proc.cat_names]
        conts = self.df.loc[i,self.proc.cont_names].values.astype(np.float32)
        return TensorTabular((tensor(cats).long(),tensor(conts).float()))
    
    def decodes(self, o) -> TabularLine:
        dic = {c: self.proc.classes[c][v] for v,c in zip(o[0], self.proc.cat_names)}
        ms = getattr(self.proc, 'means', {c:0 for c in self.proc.cont_names})
        ss = getattr(self.proc, 'stds',  {c:1 for c in self.proc.cont_names})
        dic.update({c: (v*ss[c] + ms[c]).item() for v,c in zip(o[1], self.proc.cont_names)})
        return pd.Series(dic)

In [ ]:
tds = TfmdDS([0,1,2], type_tfms=[ReadTabLine(df1, proc)])
enc = tds[1]
test_eq(enc[0][0], tensor([2,1]))
test_close(enc[0][1], tensor([2.572478]))

dec = tds.decode(enc)
assert isinstance(dec[0], TabularLine)
test_close(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 5}))

test_stdout(lambda: print(tds.show_at(1)), """a           1
b_na    False
b           5
dtype: object""")

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
df = pd.read_csv(path/'adult.csv')
cut = int(0.8*len(df))
df_trn,df_val = df.iloc[:cut],df.iloc[cut:]

In [ ]:
df1,proc = process_df(df, RandomSplitter(), procs=procs, cat_names=cat_names, cont_names=cont_names, inplace=False)

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
df1.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,education-num_na
0,0.762149,Private,-0.835340,Assoc-acdm,0.754580,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k,False
1,0.396816,Private,0.445520,Masters,1.540273,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k,False
2,-0.041584,Private,-0.883907,HS-grad,-0.031112,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k,True
3,-0.041584,Self-emp-inc,-0.726318,Prof-school,1.933119,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k,False
4,0.250683,Self-emp-not-inc,-1.015260,7th-8th,-0.031112,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k,True


In [ ]:
tfm = ReadTabLine(df1, proc)
lbl_func = lambda i: df1.salary[i]
tds = TfmdDS(range(len(df)), [tfm, [lbl_func, Categorize()]])

In [ ]:
tds.show_at(1)

workclass                    Private
education                    Masters
marital-status              Divorced
occupation           Exec-managerial
relationship           Not-in-family
race                           White
education-num_na               False
age                               44
fnlwgt                        236746
education-num                     14
category                       >=50k
dtype: object

In [ ]:
dl = TfmdDL(tds, bs=16, num_workers=0)
dl.show_batch()

index,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,category
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101319.992188,12.0,>=50k
1,Private,Masters,Divorced,Exec-managerial,Not-in-family,White,False,44.0,236746.000000,14.0,>=50k
2,Private,HS-grad,Divorced,#na#,Unmarried,Black,True,38.0,96184.992188,10.0,<50k
3,Self-emp-inc,Prof-school,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,False,38.0,112846.992188,15.0,>=50k
4,Self-emp-not-inc,7th-8th,Married-civ-spouse,Other-service,Wife,Black,True,42.0,82296.992188,10.0,<50k
5,Private,HS-grad,Never-married,Handlers-cleaners,Own-child,White,False,20.0,63209.984375,9.0,<50k
6,Private,Some-college,Divorced,#na#,Other-relative,White,False,49.0,44434.000000,10.0,<50k
7,Private,11th,Married-civ-spouse,#na#,Husband,White,False,37.0,138940.000000,7.0,<50k
8,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,46.0,328216.000000,9.0,>=50k
9,Self-emp-inc,HS-grad,Married-civ-spouse,#na#,Husband,White,True,36.0,216711.000000,10.0,>=50k


## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_script.ipynb.
Converted 02_transforms.ipynb.
Converted 03_pipeline.ipynb.
Converted 04_data_external.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 09a_rect_augment.ipynb.
Converted 11_layers.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 30_text_core.ipynb.
Converted 40_tabular_core.ipynb.
Converted 60_vision_models_xresnet.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.
Converted 95_synth_lear